# Read data example

In [1]:
import os
import typing
from tqdm import tqdm
import glob
import pandas as pd
import tensorflow as tf
import numpy as np
import json

from keras.utils import  Sequence



2025-06-13 13:02:49.341306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749819769.669135      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749819769.761008      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Read Data

In [2]:
ROOT_DATA_FOLDER = r"/kaggle/input/drive-redused"

TRAIN_DATASET_PATH = r'/kaggle/input/drive-redused/YaCupTrain'

#os.path.join(ROOT_DATA_FOLDER, r"YaCupTrain")

TEST_DATASET_PATH = os.path.join(ROOT_DATA_FOLDER, r"YaCupTest")
label_columns = ['x', 'y', 'yaw']



# Load all ids of a dataset

def read_testcase_ids(dataset_path: str):
    ids = [int(case_id) for case_id in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, case_id))]
    return ids

ids = read_testcase_ids(TRAIN_DATASET_PATH)
len(ids)

train_ids = np.random.choice(ids, size=round(0.70*len(ids)), replace=False, p=None)
test_ids = [el for el in ids if el not in train_ids]


class DataFilePaths:
    def __init__(self, testcase_path: str):
        self.testcase_path = testcase_path

    def localization(self):
        return os.path.join(self.testcase_path, 'localization.csv')

    def control(self):
        return os.path.join(self.testcase_path, 'control.csv')

    def metadata(self):
        return os.path.join(self.testcase_path, 'metadata.json')

    # exists only for test_dataset
    def requested_stamps(self):
        return os.path.join(self.testcase_path, 'requested_stamps.csv')


def read_localization(localization_path: str):
    return pd.read_csv(localization_path)

def read_control(control_path):
    return pd.read_csv(control_path)

def read_metadata(metadata_path: str):
    with open(metadata_path, 'r') as f:
        data = json.load(f)
    return data

def read_requested_stamps(requested_stamps_path: str):
    return pd.read_csv(requested_stamps_path)

def read_testcase(dataset_path: str, testcase_id: str, is_test: bool = False):
    testcase_path = os.path.join(dataset_path, str(testcase_id))
    data_file_paths = DataFilePaths(testcase_path)

    testcase_data = {}
    testcase_data['localization'] = read_localization(data_file_paths.localization())
    testcase_data['control'] = read_control(data_file_paths.control())
    testcase_data['metadata'] = read_metadata(data_file_paths.metadata())
    if is_test:
        testcase_data['requested_stamps'] = read_requested_stamps(data_file_paths.requested_stamps())

    return testcase_data


def read_testcases(dataset_path: str, is_test: bool = False, testcase_ids: typing.Iterable[int] = None):
    result = {}
    if testcase_ids is None:
        testcase_ids = read_testcase_ids(dataset_path)

    for testcase_id in tqdm(testcase_ids):
        testcase = read_testcase(dataset_path, testcase_id, is_test=is_test)
        result[testcase_id] = testcase
    return result

In [3]:
os.path.exists('/kaggle/input/drive-redused/YaCupTrain')

True

In [14]:
train_dataset[0]['localization']

,stamp_ns,x,y,z,roll,pitch,yaw
0,0,-4292.313705,-14527.266319,66.043314,0.003926,-0.054198,-1.936810
1,39989868,-4292.489928,-14527.726083,66.070022,0.003702,-0.054172,-1.936858
2,79819886,-4292.662729,-14528.183063,66.090338,0.002404,-0.054628,-1.936827
3,125154671,-4292.862032,-14528.702952,66.120814,0.002709,-0.054559,-1.936894
4,159636974,-4293.011898,-14529.097871,66.138226,0.003264,-0.053668,-1.936876
...,...,...,...,...,...,...,...
1577,63079991016,-4445.953846,-14935.006202,66.762048,0.002899,-0.001396,-1.602179
1578,63120445325,-4445.959637,-14935.233032,66.762420,0.001984,-0.001401,-1.590317
1579,63160340945,-4445.966875,-14935.455432,66.758763,-0.000359,-0.001176,-1.578275
1580,63200463435,-4445.967213,-14935.680835,66.758973,0.000547,-0.000704,-1.565992


In [4]:
train_dataset = read_testcases(TRAIN_DATASET_PATH)
len(train_dataset)

100%|██████████| 3884/3884 [01:38<00:00, 39.36it/s]


3884

In [5]:

train = {k: v for k, v in train_dataset.items() if k in train_ids}
test = {k: v for k, v in train_dataset.items() if k in test_ids}


# Window Generate

## step by step logic

In [6]:
class WindowGenerator():
  def __init__(self,train_df, input_width, label_width, shift,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df


    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])






In [7]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels


WindowGenerator.split_window = split_window

In [16]:
len(train.keys())

2719

In [22]:
train_seq_X = []
train_seq_y = []

for key in train.keys():
    w2 = WindowGenerator(train[key]['localization'], input_width=5, label_width=5, shift=1, label_columns=['x','y','yaw'])
    example_window = tf.stack([np.array(train[key]['localization'][:w2.total_window_size]),
                           np.array(train[key]['localization'][100:100+w2.total_window_size]),
                           np.array(train[key]['localization'][200:200+w2.total_window_size])
                           ])
  
    X , y = w2.split_window(example_window)

    train_seq_X.append(X)
    train_seq_y.append(y)



In [12]:
A = np.array(train_seq_X)
A.shape #[:2, :2]

(2719, 3, 5, 7)

In [31]:
train_seq_X = []
train_seq_y = []

for key in train.keys():
    w2 = WindowGenerator(train[key]['localization'], input_width=5, label_width=5, shift=1, label_columns=['x','y','yaw'])
    example_window = tf.stack([np.array(train[key]['localization'][:w2.total_window_size]),
                           np.array(train[key]['localization'][100:100+w2.total_window_size]),
                           np.array(train[key]['localization'][200:200+w2.total_window_size])
                           ])
  
    X , y = w2.split_window(example_window)

    for X_i, y_i in zip(X , y):

         train_seq_X.append(X_i)
         train_seq_y.append(y_i)
    



In [32]:
train_seq_X= np.array(train_seq_X)
train_seq_y = np.array(train_seq_y)
## Add normalize

In [33]:
train_seq_X.shape

(8157, 5, 7)

In [34]:
train_seq_X

array([[[ 0.00000000e+00, -3.03660991e+03, -1.30096477e+04, ...,
          1.57296762e-02,  1.10952043e-02,  1.20212175e+00],
        [ 4.00022710e+07, -3.03646536e+03, -1.30092770e+04, ...,
          1.53236580e-02,  1.13737995e-02,  1.20197543e+00],
        [ 8.00521900e+07, -3.03632214e+03, -1.30089065e+04, ...,
          1.57086334e-02,  1.11495178e-02,  1.20208187e+00],
        [ 1.19785340e+08, -3.03617938e+03, -1.30085350e+04, ...,
          1.66449225e-02,  1.00501508e-02,  1.20220345e+00],
        [ 1.60213741e+08, -3.03603512e+03, -1.30081616e+04, ...,
          1.71983173e-02,  1.08623850e-02,  1.20226520e+00]],

       [[ 3.99971993e+09, -3.02385700e+03, -1.29764495e+04, ...,
          2.22297988e-02,  1.58562493e-02,  1.19870945e+00],
        [ 4.03972652e+09, -3.02374734e+03, -1.29761709e+04, ...,
          2.27356715e-02,  1.57941800e-02,  1.19828144e+00],
        [ 4.08036635e+09, -3.02363673e+03, -1.29758879e+04, ...,
          2.25262258e-02,  1.55467217e-02,  1.19793

In [35]:
max_m = max(train_seq_X[:,:,0].flatten())
min_m = min(train_seq_X[:,:,0].flatten())

In [36]:
train_seq_X.shape[2]

7

## custom handwrited minmax scaller

In [37]:


for lindex in range(train_seq_X.shape[2]):
    max_m = max(train_seq_X[:,:,lindex].flatten())
    min_m = min(train_seq_X[:,:,lindex].flatten())
    for findex in range(train_seq_X.shape[0]):
        for el in range(train_seq_X.shape[1]):
            train_seq_X[findex,el,lindex] =  (train_seq_X[findex,el,lindex] -min_m)/(max_m-min_m)
        
    

In [38]:
for lindex in range(train_seq_y.shape[2]):
    max_m = max(train_seq_y[:,:,lindex].flatten())
    min_m = min(train_seq_y[:,:,lindex].flatten())
    for findex in range(train_seq_y.shape[0]):
        for el in range(train_seq_y.shape[1]):
            train_seq_y[findex,el,lindex] =  (train_seq_y[findex,el,lindex] -min_m)/(max_m-min_m)

In [39]:
max(train_seq_X.flatten())

1.0

In [40]:
min(train_seq_y.flatten())

0.0

## Metrics

Let's describe final metric. As a first step, all predicted triples $(x,y,yaw)$ are being converted into 2 points $[(x_1, y_1), (x_2, y_2)]$ in the following way:
$$
(x_1, y_1) = (x, y), \\
(x_2, y_2) = (x_1, y_1) + S \times (yaw_x, yaw_y)
$$  

where $S = 1$. In other words, we build a directed segment of length $1$. These points then used in the metric calculation.


Metric for a single pose (rmse):

$$
pose\_metric = \sqrt{ \frac{\displaystyle\sum_{j=1}^{k} {(x_j-\hat{x_j})^2 + (y_j-\hat{y_j})^2}}{k} }
$$

where $k$ - number of points that describe single pose (in our case $k=2$).

Metric for a testcase:

$$
testcase\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}pose\_metric_i
$$

where $n$ - number of localization points to predict.

And, final metric for a whole dataset:

$$
dataset\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}testcase\_metric_i
$$

where $n$ - number of test cases.


In [ ]:
import keras.backend as K

 def yandex_metrics(y_true, y_pred):
     #A[:,:,:,2]
     
     x = y_true[:,:,:,1]
     y = y_true[:,:,:,2]
     x_ =  y_pred[:,:,:,1]
     y_ =  y_pred[:,:,:,2]
     metrics = np.sqrt(np.sum((x-x_)**2 +(y-y_)**2)/2)
     #metric = np.mean(np.sqrt(2. * np.mean((points_gt - points_pred) ** 2, axis=1)))
     #metric = np.mean(np.sqrt(((x - x_) ** 2, )+(y-y_)**2))
     
     return K.mean(metric)



In [41]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(3, return_sequences=True)
    # Shape => [batch, time, features]
    #tf.keras.layers.Dense(units=5,3)
])


In [45]:
import keras

In [50]:
lstm_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SquaredHinge(),
    metrics=[
      keras.metrics.RootMeanSquaredError(), keras.metrics.CosineSimilarity()
    ])


In [ ]:
losses.Tversky()
SquaredHinge

In [51]:
lstm_model.fit(train_seq_X, train_seq_y, epochs=20, batch_size=2028)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 696ms/step - cosine_similarity: 0.7670 - loss: 0.9926 - root_mean_squared_error: 0.6312
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 653ms/step - cosine_similarity: 0.8236 - loss: 0.9904 - root_mean_squared_error: 0.6295
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 658ms/step - cosine_similarity: 0.8563 - loss: 0.9881 - root_mean_squared_error: 0.6270
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 667ms/step - cosine_similarity: 0.8753 - loss: 0.9859 - root_mean_squared_error: 0.6256
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 791ms/step - cosine_similarity: 0.8884 - loss: 0.9837 - root_mean_squared_error: 0.6233
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 666ms/step - cosine_similarity: 0.8977 - loss: 0.9814 - root_mean_squared_error: 0.6225
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 650ms/step - cosine_similarity: 0.9048 - loss: 0.9792 - root_mean_squared_error: 0.6209
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 645ms/step - cosine_similarity: 0.9101 - loss: 0.9769 - root_mean_squar

In [53]:
pr = lstm_model.predict(train_seq_X)

255/255 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step


In [56]:
train_seq_y[:,:,0]

array([[0.21922886, 0.21923326, 0.21923765, 0.21924208, 0.21924645],
       [0.21961952, 0.21962292, 0.21962621, 0.21962952, 0.2196329 ],
       [0.21997108, 0.21997485, 0.21997856, 0.21998232, 0.21998608],
       ...,
       [0.07773098, 0.07772319, 0.07771547, 0.07770778, 0.07769994],
       [0.07688105, 0.07687156, 0.07686219, 0.07685272, 0.07684318],
       [0.07582542, 0.07581437, 0.07580289, 0.07579168, 0.07578024]])

In [57]:
pr[:,:,0]

array([[0.00280817, 0.00958483, 0.0208827 , 0.03655386, 0.05599498],
       [0.00323609, 0.01106251, 0.02409751, 0.04218337, 0.06458391],
       [0.00351481, 0.01198715, 0.02605752, 0.04550077, 0.06948487],
       ...,
       [0.00310699, 0.01053451, 0.0227743 , 0.03957885, 0.06020459],
       [0.0035301 , 0.01193297, 0.02569022, 0.04447768, 0.06741444],
       [0.00381595, 0.01298584, 0.02810932, 0.0488716 , 0.07429716]],
      dtype=float32)

In [63]:
sum((train_seq_y[:,:,0].flatten() - pr[:,:,0].flatten())**2)/len(train_seq_y[:,:,0])

0.518736956832597

In [64]:
sum((train_seq_y[:,:,1].flatten() - pr[:,:,1].flatten())**2)/len(train_seq_y[:,:,1])

3.363249019276731

In [ ]:
import numpy as np
import pandas as pd

SEGMENT_LENGTH = 1.

def yaw_direction(yaw_value):
    return np.array([np.cos(yaw_value), np.sin(yaw_value)])

def build_car_points(x_y_yaw):
    directions = np.vstack(yaw_direction(x_y_yaw[:, -1]))

    front_points = x_y_yaw[:, :-1] + SEGMENT_LENGTH * directions.T
    points = np.vstack([x_y_yaw[:, :-1], front_points])
    return points

def build_car_points_from_merged_df(df: pd.DataFrame):
    points_gt = df[['x_gt', 'y_gt', 'yaw_gt']].to_numpy()
    points_pred = df[['x_pred', 'y_pred', 'yaw_pred']].to_numpy()

    points_gt = build_car_points(points_gt)
    points_pred = build_car_points(points_pred)
    return points_gt, points_pred

def calculate_metric_testcase(df: pd.DataFrame):
    points_gt, points_pred = build_car_points_from_merged_df(df)

    metric = np.mean(np.sqrt(2. * np.mean((points_gt - points_pred) ** 2, axis=1)))
    return metric

def calculate_metric_dataset(ground_truth_df: pd.DataFrame, prediction_df: pd.DataFrame):
    assert (len(ground_truth_df) == len(prediction_df))

    df = ground_truth_df.merge(prediction_df, on=['testcase_id', 'stamp_ns'], suffixes=['_gt', '_pred'])

    metric = df.groupby('testcase_id').apply(calculate_metric_testcase)
    return np.mean(metric)

In [ ]:
lstm_model.compile(train_seq_X, train_seq_y)

In [ ]:
train_seq_X = tf.zeros([5,7,1],dtype=tf.dtypes.float64,)
train_seq_y = tf.zeros([5,3,1],dtype=tf.dtypes.float64,)
print(train_seq_X.ndim, train_seq_X.shape)
for key in train.keys():
    w2 = WindowGenerator(train[key]['localization'], input_width=5, label_width=5, shift=1, label_columns=['x','y','yaw'])
    example_window = tf.stack([np.array(train[key]['localization'][:w2.total_window_size]),
                           np.array(train[key]['localization'][100:100+w2.total_window_size]),
                           np.array(train[key]['localization'][200:200+w2.total_window_size])
                           ])
  
    X , y = w2.split_window(example_window)
    #print(X)
    for X_i, y_i in zip(X , y):
        #print(X_i.ndim, X_i.shape)
        train_seq_X = tf.concat([train_seq_X,X_i], 1 )
        train_seq_y =  tf.concat([train_seq_y,y_i], 1 )



In [ ]:
train_seq_X.shape()

In [ ]:

print(train_seq_X.ndim, train_seq_X.shape)
for key in train.keys():
    w2 = WindowGenerator(train[key]['localization'], input_width=5, label_width=5, shift=1, label_columns=['x','y','yaw'])
    example_window = tf.stack([np.array(train[key]['localization'][:w2.total_window_size]),
                           np.array(train[key]['localization'][100:100+w2.total_window_size]),
                           np.array(train[key]['localization'][200:200+w2.total_window_size])
                           ])
  
    X , y = w2.split_window(example_window)
    #print(X)
    train_seq_X = tf.zeros([5,7],dtype=tf.dtypes.float64,)
    train_seq_y = tf.zeros([5,3],dtype=tf.dtypes.float64,)
    for X_i, y_i in zip(X , y):
        #print(X_i.ndim, X_i.shape)
        train_seq_X = tf.concat([train_seq_X,X_i], 0 )
        train_seq_y =  tf.concat([train_seq_y,y_i], 0 )

train_seq_X

In [ ]:
train_seq_X.shape

In [ ]:
train_seq_y

In [ ]:
train_seq_X = np.array([])

In [ ]:
train_seq_X

In [ ]:

train_seq_X = np.array([[]])
#print(train_seq_X.ndim, train_seq_X.shape)
#train_seq_y = np.array([])
for key in train.keys():
    w2 = WindowGenerator(train[key]['localization'], input_width=5, label_width=5, shift=1, label_columns=['x','y','yaw'])
    example_window = tf.stack([np.array(train[key]['localization'][:w2.total_window_size]),
                           np.array(train[key]['localization'][100:100+w2.total_window_size]),
                           np.array(train[key]['localization'][200:200+w2.total_window_size])
                           ])
  
    X , y = w2.split_window(example_window)
    #print(X)
    for X_i, y_i in zip(X , y):
        #print(X_i.ndim, X_i.shape)
        train_seq_X = np.append(train_seq_X,X_i) #,axis=0)
        #train_seq_y = np.append(train_seq_y,y_i, axis=0)

train_seq_X

In [ ]:
train_seq_X

In [ ]:
len(train_seq_y)

In [ ]:
train_seq_X

In [ ]:
w2 = WindowGenerator(train=train, val=test, label_columns=['x','y','yaw'], input_width=5, label_width=5, shift=1)

In [ ]:
w2

In [ ]:
w2 = WindowGenerator(train_df=train[3]['localization'], val_df=test[2]['localization'], label_columns=['x','y','yaw'], input_width=5, label_width=5, shift=1)

In [ ]:
example_window = tf.stack([np.array(train[3]['localization'][:w2.total_window_size]),
                           np.array(train[3]['localization'][100:100+w2.total_window_size]),
                           np.array(train[3]['localization'][200:200+w2.total_window_size])
                           ])



In [ ]:
example_inputs, example_labels = w2.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')


In [ ]:
def make_dataset(self, data):
    data = np.array(el, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
len(train)

In [ ]:
@property
def train(self):
    for i in train_ids:
        train_df = train[i]['localization']
        return self.make_dataset(self.train_df)



@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val

In [ ]:
!install paramiko

In [ ]:
wide_window = WindowGenerator( train_df=train[3]['localization'], val_df=test[2]['localization'], label_columns=['x','y','yaw'],
    input_width=24, label_width=24, shift=1,)

wide_window

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [ ]:
history = compile_and_fit(lstm_model, w2)

In [ ]:
train_dataset[5]['control']['stamp_ns'] = train_dataset[5]['control']['stamp_ns']/10e9
train_dataset[5]['localization']['stamp_ns'] = train_dataset[5]['localization']['stamp_ns']/10e9

In [ ]:
train_df = pd.DataFrame()
for el in train_dataset:
    

In [19]:
train_dataset[25]['control'].drop(columns='stamp_ns')

,acceleration_level,steering
0,-4500,-7.419072
1,-4500,-7.419072
2,-4500,-7.419072
3,-4500,-7.419072
4,-4500,-7.419072
...,...,...
1495,5969,-5.625576
1496,5911,-5.688811
1497,5816,-5.688811
1498,5674,-5.762313


In [ ]:
pd.concat([df1.set_index('A'),df2.set_index('A')], axis=1, join='inner').reset_index()


In [20]:
a = train_dataset[25]['localization'][74:1574]

In [21]:
a

,stamp_ns,x,y,z,roll,pitch,yaw
74,2960529547,148.959645,-297.067037,-21.485054,-0.008272,-0.007133,2.674813
75,3000472113,148.959645,-297.067038,-21.485054,-0.008274,-0.007132,2.674808
76,3039900228,148.959645,-297.067038,-21.485054,-0.008273,-0.007132,2.674806
77,3080404108,148.959508,-297.066878,-21.484694,-0.008322,-0.007116,2.674797
78,3119984778,148.959507,-297.066878,-21.484694,-0.008318,-0.007116,2.674795
...,...,...,...,...,...,...,...
1569,62813930520,11.228409,-297.296181,-19.815981,0.004062,-0.013882,2.460280
1570,62853584035,10.994059,-297.106072,-19.800928,0.007370,-0.018068,2.460444
1571,62893488384,10.761062,-296.917137,-19.795043,0.010270,-0.020149,2.460652
1572,62934115579,10.524072,-296.724955,-19.788326,0.016922,-0.022580,2.460336


In [ ]:
train_dataset[25]['control'].drop(columns='stamp_ns')

In [ ]:
pd.concat([train_dataset[25]['localization'][74:1574].reset_index(),train_dataset[25]['control'].drop(columns='stamp_ns').reset_index()], axis=1).reset_index().drop(columns=['index','level_0'])

In [ ]:
test_dataset = read_testcases(TEST_DATASET_PATH, is_test=True)
len(test_dataset)

In [ ]:
loc stams x y z roll pitch yaw 0--> LSTM --> next 1 loc stams x y z roll pitch yaw

In [ ]:
[el.values() for el in test_dataset.values()]

In [ ]:
# LSTM conv 1d

In [ ]:
from keras.layers.recurrent import LSTM

In [ ]:
from tensorflow.python.keras.layers.recurrent import LSTM, Bidirectional

In [ ]:
model = Sequential([
    LSTM(50, activation='tanh', return_sequences=True, input_shape=(10, 5)),  # First LSTM layer
    LSTM(30, activation='tanh'),  # Second LSTM layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
tf.keras.layers.Bidirectional(
    layer, merge_mode='concat'
)


In [ ]:
class Dataprepare():
    def __init__(dataset):
        self.dataset = dataset


    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        current_target = self.target[idx]
        current_tweet = self.tweets[idx]
        sequence = []
        for word in current_tweet:
            if word in self.word_to_idx.keys():
                sequence.append(self.word_to_idx[word])

        return {
                'x': torch.tensor(sequence, dtype=torch.long),
                'y': torch.tensor(current_target, dtype=torch.long)
            }


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

X, y = create_dataset(scaled_data)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [ ]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

## Dummy baseline

### read test dataset

In [ ]:
import numpy as np

NSECS_IN_SEC = 1000000000

def secs_to_nsecs(secs: float):
    return int(secs * NSECS_IN_SEC)

def nsecs_to_secs(nsecs: int):
    return float(nsecs) / NSECS_IN_SEC

def yaw_direction(yaw_value):
    return np.array([np.cos(yaw_value), np.sin(yaw_value)])

In [ ]:
pip install keras==2.10.0

In [ ]:
import numpy as np
np.__version__

### simple pose prediction logic without taking into account control states

In [ ]:
def localization_df_to_poses(loc, contrl):
    loc_contrl = pd.concat([loc[74:1574].reset_index(),contrl.drop(columns='stamp_ns').reset_index()], axis=1).reset_index().drop(columns=['index','level_0'])
    poses = []
    for stamp_ns, x, y, yaw,acceleration_level, steering, in zip(loc_contrl['stamp_ns'], loc_contrl['x'], loc_contrl['y'], loc_contrl['yaw'], loc_contrl['acceleration_level'],loc_contrl['steering']):
        poses.append({'stamp_ns': stamp_ns, 'pos': np.array([x, y]), 'yaw': yaw})
    return poses


In [ ]:
def localization_df_to_poses(loc_df):
    poses = []
    for stamp_ns, x, y, yaw in zip(loc_df['stamp_ns'], loc_df['x'], loc_df['y'], loc_df['yaw']):
        poses.append({'stamp_ns': stamp_ns, 'pos': np.array([x, y]), 'yaw': yaw})
    return poses

In [ ]:
localization_df_to_poses(loc_df)


In [ ]:

import numpy as np

In [ ]:
def localization_df_to_poses(loc_df):
    poses = []
    for stamp_ns, x, y, yaw in zip(loc_df['stamp_ns'], loc_df['x'], loc_df['y'], loc_df['yaw']):
        poses.append({'stamp_ns': stamp_ns, 'pos': np.array([x, y]), 'yaw': yaw})
    return poses

# naive estimation of speed at last known localization pose
def dummy_estimate_last_speed(localization_poses):
    last_pose = localization_poses[-1]

    start_pose_idx = -1
    for i, pose in enumerate(localization_poses, start=1-len(localization_poses)):
        start_pose_idx = i
        if nsecs_to_secs(last_pose['stamp_ns']) - nsecs_to_secs(pose['stamp_ns']) > 1.: # sec
            break

    start_pose = localization_poses[start_pose_idx]
    dt_sec = nsecs_to_secs(last_pose['stamp_ns']) - nsecs_to_secs(start_pose['stamp_ns'])

    if dt_sec > 1e-5:
        return np.linalg.norm(last_pose['pos'][:2] - start_pose['pos'][:2]) / dt_sec
    return 5. # some default value

def dummpy_predict_pose(last_loc_pose: dict, last_speed: float, prediction_stamp: int):
    dt_sec = nsecs_to_secs(prediction_stamp) - nsecs_to_secs(last_loc_pose['stamp_ns'])
    distance = dt_sec * last_speed
    direction = yaw_direction(last_loc_pose['yaw'])
    pos_translate = direction * distance
    return {"pos": last_loc_pose['pos'] + pos_translate, 'yaw': last_loc_pose['yaw']}

In [ ]:
def predict_testcase(testcase: dict):
    loc_df = testcase['localization']
    localization_poses = localization_df_to_poses(loc_df)

    last_loc_pose = localization_poses[-1]
    last_speed = dummy_estimate_last_speed(localization_poses)

    predicted_poses = []
    for stamp in testcase['requested_stamps']['stamp_ns']:
        pose = dummpy_predict_pose(last_loc_pose, last_speed, stamp)
        predicted_poses.append(pose)

    predictions = {}
    predictions['stamp_ns'] = testcase['requested_stamps']['stamp_ns']
    predictions['x'] = [pose['pos'][0] for pose in predicted_poses]
    predictions['y'] = [pose['pos'][1] for pose in predicted_poses]
    predictions['yaw'] = [pose['yaw'] for pose in predicted_poses]
    return pd.DataFrame(predictions)

def predict_test_dataset(test_dataset: dict):
    predictions = {}
    for testcase_id, testcase in tqdm(test_dataset.items()):
        predictions[testcase_id] = predict_testcase(testcase)
    return predictions

### make prediction for requested stamps

In [ ]:
predictions = predict_test_dataset(train_dataset)
len(predictions)

In [ ]:
test_predictions = predict_test_dataset(test_dataset)
len(test_predictions)

In [ ]:
test_predictions

### write predictions

In [ ]:
def write_predictions(dataset_predictions: dict, prediction_file_path: str):
    prediction_list = []
    for testcase_id, prediction in tqdm(dataset_predictions.items()):
        prediction['testcase_id'] = [testcase_id] * len(prediction)
        prediction_list.append(prediction)
    predictions_df = pd.concat(prediction_list)
    predictions_df = predictions_df.reindex(columns=["testcase_id", "stamp_ns", "x", "y", "yaw"])
    print(len(predictions_df))
    predictions_df.to_csv(prediction_file_path, index=False, header=True)

In [ ]:
write_predictions(test_predictions, os.path.join(ROOT_DATA_FOLDER, "predictions.csv"))

# Calculate metric

Let's describe final metric. As a first step, all predicted triples $(x,y,yaw)$ are being converted into 2 points $[(x_1, y_1), (x_2, y_2)]$ in the following way:
$$
(x_1, y_1) = (x, y), \\
(x_2, y_2) = (x_1, y_1) + S \times (yaw_x, yaw_y)
$$  

where $S = 1$. In other words, we build a directed segment of length $1$. These points then used in the metric calculation.


Metric for a single pose (rmse):

$$
pose\_metric = \sqrt{ \frac{\displaystyle\sum_{j=1}^{k} {(x_j-\hat{x_j})^2 + (y_j-\hat{y_j})^2}}{k} }
$$

where $k$ - number of points that describe single pose (in our case $k=2$).

Metric for a testcase:

$$
testcase\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}pose\_metric_i
$$

where $n$ - number of localization points to predict.

And, final metric for a whole dataset:

$$
dataset\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}testcase\_metric_i
$$

where $n$ - number of test cases.


### implementation of the metric calculation

In [ ]:
import numpy as np
import pandas as pd

SEGMENT_LENGTH = 1.

def yaw_direction(yaw_value):
    return np.array([np.cos(yaw_value), np.sin(yaw_value)])

def build_car_points(x_y_yaw):
    directions = np.vstack(yaw_direction(x_y_yaw[:, -1]))

    front_points = x_y_yaw[:, :-1] + SEGMENT_LENGTH * directions.T
    points = np.vstack([x_y_yaw[:, :-1], front_points])
    return points

def build_car_points_from_merged_df(df: pd.DataFrame):
    points_gt = df[['x_gt', 'y_gt', 'yaw_gt']].to_numpy()
    points_pred = df[['x_pred', 'y_pred', 'yaw_pred']].to_numpy()

    points_gt = build_car_points(points_gt)
    points_pred = build_car_points(points_pred)
    return points_gt, points_pred

def calculate_metric_testcase(df: pd.DataFrame):
    points_gt, points_pred = build_car_points_from_merged_df(df)

    metric = np.mean(np.sqrt(2. * np.mean((points_gt - points_pred) ** 2, axis=1)))
    return metric

def calculate_metric_dataset(ground_truth_df: pd.DataFrame, prediction_df: pd.DataFrame):
    assert (len(ground_truth_df) == len(prediction_df))

    df = ground_truth_df.merge(prediction_df, on=['testcase_id', 'stamp_ns'], suffixes=['_gt', '_pred'])

    metric = df.groupby('testcase_id').apply(calculate_metric_testcase)
    return np.mean(metric)